In [2]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from scipy.spatial.distance import euclidean

from mode_imputer import KNNImputerMode

In [3]:
data = pd.read_csv('./Financial Well-Being Survey Data/NFWBS_PUF_2016_data.csv')

print(f'The dataset contains {data.shape[0]} rows and {data.shape[1]} features.')

The dataset contains 6394 rows and 217 features.


# Data cleaning

In [4]:
data.set_index('PUF_ID', inplace=True)

## Dropping irrelevant features

We can remove features that were used to feature engineering

Engineered features: 
- FWBScore: created from FWB questions
- FSscore: created from FS questions
- LMscore: created from FINKKNOWL questions
- KHscore: created from KHKNOWL questions

In [5]:
FWB_cols = list(filter(lambda x: 'FWB' in x, data.columns.values))

FWB_cols

['FWBscore',
 'FWB1_1',
 'FWB1_2',
 'FWB1_3',
 'FWB1_4',
 'FWB1_5',
 'FWB1_6',
 'FWB2_1',
 'FWB2_2',
 'FWB2_3',
 'FWB2_4']

In [6]:
FWB_drop = FWB_cols[1:]

FWB_drop

['FWB1_1',
 'FWB1_2',
 'FWB1_3',
 'FWB1_4',
 'FWB1_5',
 'FWB1_6',
 'FWB2_1',
 'FWB2_2',
 'FWB2_3',
 'FWB2_4']

In [7]:
data.drop(FWB_drop, axis = 1, inplace = True)

In [8]:
list(filter(lambda x: 'FWB' in x, data.columns.values))

['FWBscore']

In [9]:
FS_cols = list(filter(lambda x: 'FS' in x, data.columns.values))

FS_cols = FS_cols[1:]

data.drop(FS_cols, axis = 1, inplace = True)

list(filter(lambda x: 'FS' in x, data.columns.values))

['FSscore']

In [10]:
LM_cols = list(filter(lambda x: 'FINKNOWL' in x, data.columns.values))

data.drop(LM_cols, axis = 1, inplace = True)

list(filter(lambda x: 'FINKNOWL' in x, data.columns.values))

[]

In [11]:
LM_cols = list(filter(lambda x: 'KHKNOWL' in x, data.columns.values))

data.drop(LM_cols, axis = 1, inplace = True)

list(filter(lambda x: 'KHKNOWL' in x, data.columns.values))

[]

In [12]:
data.shape

(6394, 184)

## Missing values

__-5: County not known__


Use PPREG9, PPINCIMP and PPEDUC to impute PCTLT200FPL

PPREG9: categorical
<br>
PPINCIMP: ordinal
<br>
PPEDUC: ordinal

distance to find nearest neighbors: euclidean for PPINC, PPEDUC  + one hot for PPREG9

In [13]:
# replace by missing values
data = data.replace(-5, np.nan)

In [14]:
data[data['PCTLT200FPL'].isna()]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
11615,3,3,5,4,5,59,38,5,5,4,...,1,4,8,0,0,0,0,1,NaN,0.749584
14199,3,2,3,4,6,43,53,5,5,4,...,1,3,7,0,0,0,0,3,NaN,1.504086
11487,3,3,7,7,7,57,59,5,4,4,...,1,3,6,0,0,0,0,2,NaN,0.844349
7194,3,3,6,6,7,41,49,6,4,4,...,1,3,5,0,0,0,0,2,NaN,0.649568
13757,3,1,5,7,7,57,61,5,5,5,...,1,4,9,0,0,0,0,4,NaN,0.785893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13144,3,2,6,7,7,58,63,6,4,4,...,1,3,5,0,0,0,0,1,NaN,1.577823
11620,3,2,7,6,7,44,45,5,4,3,...,1,4,9,0,1,1,0,4,NaN,2.563018
11220,3,3,6,7,7,61,47,5,5,4,...,1,2,3,0,0,0,1,2,NaN,0.522504


In [15]:
print(f'Index of PPEDUC: {list(data.columns.values).index("PPEDUC")}')
print(f'Index of PPINCIMP: {list(data.columns.values).index("PPINCIMP")}')
print(f'Index of PPREG9: {list(data.columns.values).index("PPREG9")}')

Index of PPEDUC: 168
Index of PPINCIMP: 172
Index of PPREG9: 176


In [16]:
def distance(X, Y, missing_values = np.nan):
    # X = (217, )
    # Y = (217, )

    eucl_dist = euclidean(X[[168, 172]], Y[[168, 172]])

    return eucl_dist + (X[176] != Y[176])

In [17]:
imputer = KNNImputer(n_neighbors = 5, metric = distance)

new_data = imputer.fit_transform(data)

data = pd.DataFrame(new_data, columns = data.columns, index = data.index)

Given the variable is binary and it was imputed using the mean of the 5 nearest neighbours we know that if 3/5 of the neighbours are 1, then the imputed value is 0.6 any value above this threshold is considered 1 any value below this threshold is considered 0

So, we need to convert 0.6, 0.8 to 1 and 0.2, 0.4 to 0

In [18]:
data['PCTLT200FPL'].value_counts()

0.0    4636
1.0    1460
0.2     133
0.6      75
0.4      73
0.8      17
Name: PCTLT200FPL, dtype: int64

In [19]:
data.loc[data['PCTLT200FPL'] < 0.6, 'PCTLT200FPL'] = 0
data.loc[data['PCTLT200FPL'] >= 0.6, 'PCTLT200FPL'] = 1

In [20]:
data['PCTLT200FPL'].value_counts()

0.0    4842
1.0    1552
Name: PCTLT200FPL, dtype: int64

__-4: Response not written to the database due to error__: SWB only (because FWB dropped) ->DROP

In [21]:
data.drop(12173, axis = 0, inplace = True)

__-3: Invalid response/ Incoherent data__

Let's drop the only row with KIDS_2 = -3 and let's use agecat, PPEDUC, PPINCIMP, PPMARIT and AUTOMATED_1 to impute SOCSEC2 and SOCSEC3

agecat: ordinal
<br>
PPEDUC: ordinal
<br>
PPINCIMP: ordinal
<br>
PPMARIT: categorical
<br>
AUTOMATED_1: categorical

distance: euclidean for agecat, PPEDUC and PPINCIMP + hamming for PPMARIT and AUTOMATED_1

KIDS_2 drop

In [22]:
data[data['KIDS_2'] == -3]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
13237,1.0,2.0,3.0,7.0,7.0,52.0,50.0,5.0,4.0,4.0,...,1.0,2.0,3.0,0.0,0.0,1.0,1.0,3.0,0.0,1.690435


In [23]:
data.drop(13237, axis = 0, inplace = True)

SOCSEC2 imputation

In [24]:
data[data['SOCSEC2'] == -3]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
12675,1.0,3.0,7.0,7.0,7.0,64.0,68.0,6.0,5.0,4.0,...,1.0,3.0,7.0,0.0,0.0,0.0,1.0,4.0,1.0,0.369340
9182,1.0,3.0,6.0,-1.0,-1.0,50.0,43.0,4.0,4.0,3.0,...,1.0,4.0,9.0,0.0,0.0,0.0,0.0,2.0,0.0,0.647301
10550,2.0,3.0,6.0,5.0,4.0,50.0,37.0,4.0,3.0,3.0,...,1.0,4.0,9.0,0.0,0.0,0.0,0.0,3.0,1.0,0.279867
11083,1.0,1.0,3.0,4.0,3.0,38.0,40.0,5.0,-1.0,4.0,...,1.0,4.0,9.0,0.0,0.0,0.0,1.0,4.0,1.0,1.116788
9185,1.0,2.0,5.0,4.0,4.0,47.0,41.0,4.0,3.0,3.0,...,1.0,3.0,7.0,0.0,0.0,0.0,0.0,3.0,1.0,1.146205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12089,3.0,1.0,1.0,3.0,1.0,48.0,43.0,4.0,4.0,3.0,...,0.0,3.0,6.0,0.0,0.0,0.0,0.0,1.0,1.0,0.857919
13848,3.0,1.0,5.0,5.0,7.0,18.0,41.0,3.0,5.0,3.0,...,1.0,3.0,7.0,0.0,0.0,0.0,0.0,3.0,1.0,1.444869
12971,3.0,3.0,4.0,5.0,2.0,52.0,43.0,5.0,-1.0,2.0,...,1.0,4.0,9.0,0.0,0.0,0.0,0.0,1.0,0.0,0.545312


In [25]:
data['SOCSEC2'] = data['SOCSEC2'].replace(-3, np.nan)

In [26]:
data['SOCSEC2'].value_counts()

-2.0     4526
 62.0     739
 65.0     459
 66.0     159
 63.0     130
 64.0      68
 67.0      55
 70.0      51
 68.0      19
-1.0       12
 69.0       8
Name: SOCSEC2, dtype: int64

We have a lot of rows without an answer to 'SOCSEC2', therefore we do not want to include these rows in the valid neighbours to impute the data

In [27]:
valid_neigbors = data[(data['SOCSEC2'] != -1) & (data['SOCSEC2'] != -2)]

In [28]:
valid_neigbors['SOCSEC2'].value_counts()

62.0    739
65.0    459
66.0    159
63.0    130
64.0     68
67.0     55
70.0     51
68.0     19
69.0      8
Name: SOCSEC2, dtype: int64

In [29]:
valid_neigbors['SOCSEC2'].isna().sum()

166

In [30]:
print(f'Index of agecat: {list(data.columns.values).index("agecat")}')
print(f'Index of PPEDUC: {list(data.columns.values).index("PPEDUC")}')
print(f'Index of PPINCIMP: {list(data.columns.values).index("PPINCIMP")}')
print(f'Index of PPMARIT: {list(data.columns.values).index("PPMARIT")}')
print(f'Index of AUTOMATED_1: {list(data.columns.values).index("AUTOMATED_1")}')

Index of agecat: 166
Index of PPEDUC: 168
Index of PPINCIMP: 172
Index of PPMARIT: 173
Index of AUTOMATED_1: 21


In [31]:
def distance(X, Y, missing_values = np.nan):
    # X = (217, )
    # Y = (217, )

    eucl_dist = euclidean(X[[166, 168, 172]], Y[[166, 168, 172]])

    return eucl_dist + (X[173] != Y[173]) + (X[21] != Y[21])

In [32]:
imputer = KNNImputer(n_neighbors = 5, metric = distance)

new_data = imputer.fit_transform(valid_neigbors)

valid_neigbors = pd.DataFrame(new_data, columns = data.columns, index = valid_neigbors.index)

In [33]:
valid_neigbors['SOCSEC2'].value_counts() # there are ages that need to be rounded

62.0    745
65.0    466
66.0    159
63.0    141
64.0     74
67.0     55
70.0     51
63.4     27
68.0     19
63.2     17
62.8     17
64.2     11
62.6     11
64.6      9
63.6      9
69.0      8
64.8      6
64.4      6
63.8      6
62.2      5
65.2      5
66.2      2
62.4      2
66.6      1
65.4      1
66.4      1
Name: SOCSEC2, dtype: int64

In [34]:
valid_neigbors.loc[:, 'SOCSEC2'] = valid_neigbors.loc[:, 'SOCSEC2'].round()

In [35]:
valid_neigbors['SOCSEC2'].value_counts()

62.0    752
65.0    487
63.0    213
66.0    162
64.0    106
67.0     56
70.0     51
68.0     19
69.0      8
Name: SOCSEC2, dtype: int64

In [36]:
data.loc[data['SOCSEC2'].isna(), 'SOCSEC2'] = valid_neigbors.loc[data['SOCSEC2'].isna(), 'SOCSEC2'].copy()

In [37]:
data['SOCSEC2'].value_counts()

-2.0     4526
 62.0     752
 65.0     487
 63.0     213
 66.0     162
 64.0     106
 67.0      56
 70.0      51
 68.0      19
-1.0       12
 69.0       8
Name: SOCSEC2, dtype: int64

In [38]:
data.loc[12675,'SOCSEC2']

63.0

SOCSEC3 imputation

In [39]:
data[data['SOCSEC3'] == -3]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
11621,1.0,3.0,5.0,6.0,7.0,49.0,36.0,5.0,5.0,3.0,...,1.0,2.0,4.0,0.0,0.0,0.0,0.0,2.0,0.0,0.909189
12358,1.0,3.0,5.0,4.0,7.0,46.0,51.0,4.0,5.0,4.0,...,1.0,4.0,9.0,0.0,0.0,0.0,0.0,3.0,1.0,0.504502
13446,1.0,2.0,7.0,6.0,1.0,51.0,85.0,2.0,5.0,3.0,...,1.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.770960
10138,1.0,1.0,1.0,1.0,1.0,39.0,79.0,7.0,5.0,5.0,...,0.0,3.0,6.0,0.0,0.0,0.0,0.0,1.0,1.0,1.063560
8733,1.0,1.0,6.0,7.0,5.0,35.0,62.0,5.0,4.0,5.0,...,1.0,3.0,5.0,0.0,0.0,0.0,1.0,1.0,1.0,1.196228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12514,3.0,3.0,7.0,7.0,7.0,86.0,85.0,7.0,5.0,5.0,...,1.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,1.0,1.595721
14121,3.0,2.0,4.0,7.0,7.0,52.0,70.0,7.0,5.0,5.0,...,1.0,3.0,7.0,0.0,0.0,0.0,0.0,1.0,1.0,1.573669
12988,3.0,1.0,7.0,7.0,7.0,55.0,48.0,5.0,4.0,3.0,...,0.0,4.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,3.037135


In [40]:
data['SOCSEC3'] = data['SOCSEC3'].replace(-3, np.nan)

In [41]:
data['SOCSEC3'].value_counts()

-2.0     2253
 65.0    1221
 70.0     723
 67.0     451
 61.0     406
 62.0     372
 71.0     364
 68.0     160
-1.0      140
 66.0     118
 63.0      51
 69.0      41
 64.0      24
Name: SOCSEC3, dtype: int64

In [42]:
valid_neigbors = data[(data['SOCSEC3'] != -1) & (data['SOCSEC3'] != -2)]

In [43]:
valid_neigbors['SOCSEC3'].value_counts()

65.0    1221
70.0     723
67.0     451
61.0     406
62.0     372
71.0     364
68.0     160
66.0     118
63.0      51
69.0      41
64.0      24
Name: SOCSEC3, dtype: int64

In [44]:
imputer = KNNImputer(n_neighbors = 5, metric = distance)

new_data = imputer.fit_transform(valid_neigbors)

valid_neigbors = pd.DataFrame(new_data, columns = data.columns, index = valid_neigbors.index)

In [45]:
valid_neigbors.loc[:, 'SOCSEC3'] = valid_neigbors.loc[:, 'SOCSEC3'].round()

valid_neigbors['SOCSEC3'].value_counts()

65.0    1234
70.0     724
67.0     460
61.0     406
62.0     373
71.0     364
68.0     162
66.0     136
63.0      58
69.0      43
64.0      39
Name: SOCSEC3, dtype: int64

In [46]:
data.loc[data['SOCSEC3'].isna(), 'SOCSEC3'] = valid_neigbors.loc[data['SOCSEC3'].isna(), 'SOCSEC3'].copy()

In [47]:
data['SOCSEC3'].value_counts()

-2.0     2253
 65.0    1234
 70.0     724
 67.0     460
 61.0     406
 62.0     373
 71.0     364
 68.0     162
-1.0      140
 66.0     136
 63.0      58
 69.0      43
 64.0      39
Name: SOCSEC3, dtype: int64

__-1: Not answered/ Left blank__

- SWB: median
- FWBscore: drop
- FSscore: drop
- SUBKNOWL1: median
- ACT:
- FINGOALS:
- PROPLAN:
- MANAGE:
- SAVEHABIT:
- FRUGALITY:
- AUTOMATED:
- ASK:
- SUBNUMERACY:
- CHANGEABLE:
- GOALCONF:
- ENDSMEET:
- HOUSING:
- LIVINGARRANGEMENT:
- HOUSERANGES:
- VALUERANGES:
- MORTAGE:
- SAVINGSRANGES:
- CONSPROTECT:
- EARNERS:
- VOLATILITY:
- SNAP:
- MATHARDSHIP:
- COLLECT:
- REJECTED:
- ABSORBSHOCK:
- BENEFITS:
- FRAUD2:
- COVERCOSTS:
- BORROW:
- MANAGE2:
- PAIDHELP:
- HSLOC:
- PAREDUC:
- FINSOC:
- OBJNUMERACY1:
- MATERIALISM:
- CONNECT:
- HEALTH:
- SCFHORIZON:
- DISCOUNT:
- MEMLOSS:
- DISTRESS:
- SELFCONTROL:
- OUTLOOK:
- PEM:
- HOUSESAT:
- SOCSEC:
- LIFEEXPECT:
- HHEDUC:
- KIDS:
- RETIRE:
- MILITARY:
- Military_Status:

First, let's replace the missing values with NaN

In [48]:
data = data.replace(-1, np.nan)

Now, let's see which columns have missing values

In [49]:
data.columns[data.isna().sum() > 0]

Index(['SWB_1', 'SWB_2', 'SWB_3', 'FWBscore', 'FSscore', 'SUBKNOWL1', 'ACT1_1',
       'ACT1_2', 'FINGOALS', 'PROPPLAN_1', 'PROPPLAN_2', 'PROPPLAN_3',
       'PROPPLAN_4', 'MANAGE1_1', 'MANAGE1_2', 'MANAGE1_3', 'MANAGE1_4',
       'SAVEHABIT', 'FRUGALITY', 'AUTOMATED_1', 'AUTOMATED_2', 'ASK1_1',
       'ASK1_2', 'SUBNUMERACY2', 'SUBNUMERACY1', 'CHANGEABLE', 'GOALCONF',
       'ENDSMEET', 'HOUSING', 'LIVINGARRANGEMENT', 'HOUSERANGES',
       'VALUERANGES', 'MORTGAGE', 'SAVINGSRANGES', 'CONSPROTECT1',
       'CONSPROTECT2', 'CONSPROTECT3', 'EARNERS', 'VOLATILITY', 'SNAP',
       'MATHARDSHIP_1', 'MATHARDSHIP_2', 'MATHARDSHIP_3', 'MATHARDSHIP_4',
       'MATHARDSHIP_5', 'MATHARDSHIP_6', 'COLLECT', 'REJECTED_1', 'REJECTED_2',
       'ABSORBSHOCK', 'BENEFITS_1', 'BENEFITS_2', 'BENEFITS_3', 'BENEFITS_4',
       'BENEFITS_5', 'FRAUD2', 'COVERCOSTS', 'BORROW_1', 'BORROW_2', 'MANAGE2',
       'PAIDHELP', 'HSLOC', 'PAREDUC', 'FINSOC2_1', 'FINSOC2_2', 'FINSOC2_3',
       'FINSOC2_4', 'FINSOC2_5',

__SBW: Subjective well-being and optimism questions__

In [50]:
data.loc[data['SWB_1'].isna(), 'SWB_1'] = data['SWB_1'].median()
data.loc[data['SWB_2'].isna(), 'SWB_2'] = data['SWB_2'].median()
data.loc[data['SWB_3'].isna(), 'SWB_3'] = data['SWB_3'].median()

__FWBscore__

In [51]:
data.drop(data[data['FWBscore'].isna()].index, inplace = True)

__FSscore: Financial skill scale score__

In [52]:
data.drop(data[data['FSscore'].isna()].index, inplace = True)

__SUBKNOWL1: How would you assess your overall financial knowledge?__

In [53]:
data.loc[data['SUBKNOWL1'].isna(), 'SUBKNOWL1'] = data['SUBKNOWL1'].median()

__ACT__

ACT1_1: KNNImputer using FINGOALS, PROPPLAN_1, MANAGE1_1 and ASK1_1
<br>
ACT1_2: KNNImputer using FINGOALS, PORPPLAN_1, MANAGE1_2 and ASK1_1

FINGOALS: binary
PROPPLAN_1: ordinal
MANAGE1_1: ordinal
ASK_1: ordinal

distance: euclidean

use KNeighborsClassifier -> remove the rows with missing values in FINGOALS, PROPPLAN_1, MANAGE1_1 and ASK1_1
<br>
ask chat what to do

In [54]:
data[list(filter(lambda x: 'ACT' in x, data.columns.values))].isna().sum()

ACT1_1    20
ACT1_2    14
dtype: int64

In [55]:
data.loc[data['ACT1_1'].isna(), ['FINGOALS', 'PROPPLAN_1', 'MANAGE1_1' ,'ASK1_1', 'ACT1_1']]

,FINGOALS,PROPPLAN_1,MANAGE1_1,ASK1_1,ACT1_1
PUF_ID,,,,,
7267,NaN,3.0,4.0,NaN,NaN
11083,0.0,4.0,4.0,5.0,NaN
9698,0.0,4.0,4.0,4.0,NaN
9719,NaN,2.0,5.0,2.0,NaN
13130,0.0,3.0,2.0,3.0,NaN
9752,1.0,4.0,5.0,5.0,NaN
12998,1.0,4.0,5.0,4.0,NaN
7214,NaN,3.0,3.0,NaN,NaN
13517,1.0,5.0,5.0,5.0,NaN


There is one row with missing values in all columns, so let's drop it given it is not possible to infer the value of ACT1_1.

In [56]:
data.drop(7198, inplace = True)

In [57]:
data.loc[data['ACT1_2'].isna(), ['FINGOALS', 'PROPPLAN_1', 'MANAGE1_2' ,'ASK1_1', 'ACT1_2']]

,FINGOALS,PROPPLAN_1,MANAGE1_2,ASK1_1,ACT1_2
PUF_ID,,,,,
7267,NaN,3.0,4.0,NaN,NaN
9698,0.0,4.0,2.0,4.0,NaN
9719,NaN,2.0,2.0,2.0,NaN
12247,1.0,5.0,5.0,4.0,NaN
9785,1.0,5.0,5.0,3.0,NaN
9072,0.0,4.0,4.0,3.0,NaN
9815,1.0,4.0,3.0,4.0,NaN
7214,NaN,3.0,3.0,NaN,NaN
13166,NaN,3.0,NaN,5.0,NaN


In [58]:
imputer = KNNImputerMode(n_neighbors = 5)

new_data = imputer.fit_transform(data[['FINGOALS', 'PROPPLAN_1', 'MANAGE1_1' ,'ASK1_1', 'ACT1_1']])

new_data = pd.DataFrame(new_data, columns = ['FINGOALS', 'PROPPLAN_1', 'MANAGE1_1' ,'ASK1_1', 'ACT1_1'], index = data.index)

In [59]:
new_data.loc[data[data['ACT1_1'].isna()].index.values, 'ACT1_1']

PUF_ID
7267     4.0
11083    4.0
9698     4.0
9719     5.0
13130    3.0
9752     5.0
12998    5.0
7214     3.0
13517    5.0
13937    5.0
10300    1.0
11807    3.0
11328    4.0
11712    4.0
11614    3.0
12230    4.0
7988     3.0
12934    4.0
12971    3.0
Name: ACT1_1, dtype: float64

In [60]:
data.loc[data['ACT1_1'].isna(), 'ACT1_1'] = new_data.loc[data['ACT1_1'].isna(), 'ACT1_1'].copy()

In [61]:
imputer = KNNImputerMode(n_neighbors = 5)

new_data = imputer.fit_transform(data[['FINGOALS', 'PROPPLAN_1', 'MANAGE1_2' ,'ASK1_1', 'ACT1_2']])

new_data = pd.DataFrame(new_data, columns = ['FINGOALS', 'PROPPLAN_1', 'MANAGE1_2' ,'ASK1_1', 'ACT1_2'], index = data.index)

In [62]:
data.loc[data['ACT1_2'].isna(), 'ACT1_2'] = new_data.loc[data['ACT1_2'].isna(), 'ACT1_2'].copy()

In [63]:
data[list(filter(lambda x: 'ACT' in x, data.columns.values))].isna().sum()

ACT1_1    0
ACT1_2    0
dtype: int64

__FINGOALS: Do you have a current or recent financial goal?__

Let's use ACT1_2, SAVEHABIT, PROPPLAN_3 and GOALCONF to find the K nearest neighbors to impute the missing values

ACT1_2: ordinal
<br>
SAVEHABIT: ordinal
<br>
PROPPLAN_3: ordinal
<br>
GOALCONF: ordinal

distance: euclidean

In [64]:
data[data['FINGOALS'].isna()]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
7267,1.0,3.0,6.0,6.0,6.0,49.0,42.0,5.0,4.0,4.0,...,1.0,3.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,1.410871
13389,1.0,3.0,4.0,4.0,4.0,51.0,41.0,5.0,3.0,3.0,...,1.0,4.0,9.0,0.0,0.0,0.0,1.0,4.0,0.0,1.163321
9916,1.0,3.0,7.0,6.0,6.0,62.0,46.0,4.0,5.0,3.0,...,1.0,3.0,5.0,0.0,0.0,0.0,1.0,2.0,0.0,0.637269
13943,1.0,3.0,6.0,4.0,4.0,44.0,34.0,3.0,4.0,2.0,...,1.0,2.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.965428
13452,2.0,3.0,6.0,5.0,4.0,64.0,49.0,6.0,4.0,4.0,...,1.0,2.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.528451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7369,3.0,3.0,7.0,7.0,7.0,55.0,62.0,7.0,5.0,5.0,...,0.0,3.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.736764
13807,3.0,3.0,7.0,7.0,7.0,46.0,49.0,5.0,4.0,3.0,...,1.0,2.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.501238
10344,3.0,2.0,3.0,6.0,2.0,43.0,36.0,2.0,4.0,2.0,...,1.0,3.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,1.356777


In [67]:
data['FINGOALS'].value_counts()

1.0    4046
0.0    2265
Name: FINGOALS, dtype: int64

In [65]:
imputer = KNNImputerMode(n_neighbors = 5)

new_data = imputer.fit_transform(data[['FINGOALS', 'ACT1_2', 'SAVEHABIT' ,'PROPPLAN_3', 'GOALCONF']])

new_data = pd.DataFrame(new_data, columns = ['FINGOALS', 'ACT1_2', 'SAVEHABIT' ,'PROPPLAN_3', 'GOALCONF'], index = data.index)

In [66]:
new_data['FINGOALS'].value_counts()

1.0    4093
0.0    2290
Name: FINGOALS, dtype: int64

In [68]:
data.loc[data['FINGOALS'].isna(), 'FINGOALS'] = new_data.loc[data['FINGOALS'].isna(), 'FINGOALS'].copy()

In [69]:
data['FINGOALS'].isna().sum()

0